# Introduction

This notebook is based on the code of [ConservFold](https://www.rodrigueslab.com/resources). The input data that you need to upload is:
1. The AlphaFold2 (AF2) model in .pdb format
2. A set of custom homologous sequences of interest in .fasta format

This notebook will take these inputs and compute a custom MSA to detect the most conserved amino acids in relation to the sequence from the AF2 model. Then, from this MSA compute the Weblogo entropy (AKA conservation score) and parse it into the AF2 model.


* Written by GAMA ([@miangoar on Twitter](https://twitter.com/miangoar))
* Date: 04/2024

In [ ]:
#@title 1) Install libraries
# @markdown
! pip install -Uqqq weblogo
! pip install -Uqqq biopython
! pip install -Uqqq pdb-tools

# seqkit install
! curl -s -O -L https://github.com/shenwei356/seqkit/releases/download/v2.8.0/seqkit_linux_amd64.tar.gz
! tar -xzvf seqkit_linux_amd64.tar.gz
! cp seqkit /usr/local/bin/

# mafft install
! wget -qO ac.sh https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
! bash ./ac.sh -bfp /usr/local
! conda install bioconda::mafft -y

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.7/571.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.0 MB/s eta 0:00:00
seqkit
PREFIX=/usr/local
Unpacking payload ...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /usr/local
Channels:
 - defaults
 - bioconda
Platform: linux-64
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - bioconda::mafft


The following packages will be downloaded:

    

In [ ]:
#@title 2) Import libraries
import pandas as pd
import numpy as np
import os

from google.colab import files

from Bio.PDB import PDBParser, PDBIO
from Bio.PDB.StructureBuilder import StructureBuilder

In [ ]:
#@title 3) Upload your AF2 model (in .pdb format)
# @markdown

# get the original name and rename it
upload = files.upload()
upload_file = list(upload.keys())[0]
os.rename(upload_file, "af2_model.pdb")

Saving my_af2_model.pdb to my_af2_model.pdb


In [ ]:
#@title 4) Upload your custom homologous sequences (in .fasta format)

uploaded = files.upload()

# get the original name and rename it
uploaded_filename = list(uploaded.keys())[0]
os.rename(uploaded_filename, "my_homologous_sequences.fasta")

Saving my_homologous_sequences.fasta to my_homologous_sequences.fasta


In [ ]:
#@title 5) Compute the MSA

# extract the seq from the af2 model
! pdb_tofasta af2_model.pdb > af2_sequence.fasta
! sed -i 's/^>[^ ]*/>af2_seq/' af2_sequence.fasta

# concat the data
! cat af2_sequence.fasta  my_homologous_sequences.fasta > input_sequences.fasta

# rename the headers
! seqkit replace -p "\t| " -r '_' input_sequences.fasta  > sequences.fasta

# run mafft
! mafft --quiet sequences.fasta > msa.fasta

# convert the msa to csv
! seqkit fx2tab msa.fasta > msa.csv

# create a df from the msa
df = pd.read_csv("msa.csv", sep="\t",names=["seq_id","seq"], usecols=[0,1])
df.set_index('seq_id', inplace=True)

# expand the df as a column per character in the msa
new_df = pd.DataFrame(df['seq'].apply(list).tolist(), index=df.index)

# extract the index that correspond to the AF2 seq
af2_seq_data = new_df.loc["af2_seq"]

# detect the gapped columns in the df
gaps = [idx for idx, val in af2_seq_data.items() if val == '-']

# remove the gapped columns adn reformat the df
clean = new_df.drop(columns=gaps, errors='ignore')
clean['seq'] = clean.apply(lambda row: ''.join(row.astype(str)), axis=1)
clean = clean[['seq']]
clean.reset_index(inplace=True)
clean.rename(columns={'index': clean.index.name}, inplace=False)

# a fx to export fasta seqs
def export_fasta(df, output_file):
    with open(output_file, 'w') as f:
        for index, row in df.iterrows():
            f.write(f'>{row["seq_id"]}\n')
            f.write(f'{row["seq"]}\n')

# export the cleaned df as a fasta file
export_fasta(clean, "parsed_msa.fasta")

In [ ]:
#@title 6) PDB parser
# @markdown This will download the file "af2_model_weblogo.pdb" that is the PDB file with the conservation score in the B-factor column.

# @markdown Aditionally will download the file "parsed_msa.fasta" that is the custom MSA used to compute the conservation scores. This MSA can be visualized with tools like [ugene](https://ugene.net/).

# compute the entropy from the msa
!weblogo --format logodata < parsed_msa.fasta > weblogo.txt

pdb_filename = f'af2_model.pdb' # the mode that comes from AF2
weblogo_filename = 'weblogo.txt'

# Read entropy values from weblogo.txt
entropy_values = []
with open(weblogo_filename, 'r') as weblogo_file:
    for line in weblogo_file:
        if not line.startswith("#"):
            data = line.split()
            entropy = float(data[-4])  # Extract entropy value (4th from the end)
            entropy_values.append(entropy)

# Read the PDB file
parser = PDBParser(QUIET=True)
structure = parser.get_structure('protein', pdb_filename)

# Modify the beta factor of atoms based on entropy values
for model in structure:
    for chain in model:
        for residue in chain:
            for atom in residue:
                residue_index = residue.id[1] - 1  # Adjust for 0-based index
                if residue_index < len(entropy_values):
                    beta_factor = entropy_values[residue_index] ** 2  # Square the entropy value
                    atom.set_bfactor(beta_factor)

# Save the modified PDB file
output_filename = pdb_filename.replace('af2_model.pdb', 'af2_model_weblogo.pdb')
io = PDBIO()
io.set_structure(structure)
io.save(output_filename)

# download the annotated pdb
files.download('af2_model_weblogo.pdb')
files.download('parsed_msa.fasta')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title (Optional) Set a function to see the most conserved position
# @markdown This function takes the computed MSA and create a csv file in order to find the most conserved positions given a threshold (as percent 1 to 100)

def find_conserved_positions(df, conservation_threshold):
    # Convert the sequence into a character matrix
    char_matrix = np.array([list(seq) for seq in df['seq']])

    # Get the dimensions of the matrix
    rows, cols = char_matrix.shape

    # Create a DataFrame from the matrix
    df_chars = pd.DataFrame(char_matrix, columns=[f'position_{i+1}' for i in range(cols)])

    # Calculate the total number of rows in each column
    total_rows = df_chars.apply(lambda x: x.count())

    # Initialize a dictionary to store the results
    result_dict = {}

    # Iterate over each column of the DataFrame
    for col in df_chars.columns:
        # Calculate the frequency of each character in the current column
        char_counts = df_chars[col].value_counts().to_dict()
        # Calculate the conservation percentage for each character
        conservation_percentages = {char: count / total_rows[col] * 100 for char, count in char_counts.items()}
        # Add the results to the final dictionary
        result_dict[col] = conservation_percentages

    # Filter the columns that have at least one character with a conservation percentage equal to or greater than the conservation threshold
    filtered_result = {}
    for col, percentages in result_dict.items():
        non_dash_percentages = {char: percentage for char, percentage in percentages.items() if char != "-"}
        if any(percentage >= conservation_threshold for percentage in non_dash_percentages.values()):
            filtered_result[col] = non_dash_percentages

    # Create a dictionary with the conserved positions with a percentage equal to or greater than the conservation threshold
    conserved_positions = {}
    for col, percentages in filtered_result.items():
        conserved_chars = {char: percentage for char, percentage in percentages.items() if percentage >= conservation_threshold}
        if conserved_chars:
            conserved_positions[col] = conserved_chars

    return conserved_positions

In [ ]:
#@title (Optional) Run the function
# @markdown Set the conservation threshold that you want. It will show the specific positions that passed the threshold.

conservation_threshold = "90" #@param {type:"string"}
conserved_positions = find_conserved_positions(clean, int(conservation_threshold))
print(f"Positions conserved above {conservation_threshold}%")
conserved_positions

Positions conserved above 90%


{'position_165': {'E': 95.1219512195122},
 'position_169': {'V': 92.6829268292683},
 'position_171': {'R': 92.6829268292683},
 'position_175': {'L': 95.1219512195122},
 'position_180': {'H': 93.90243902439023},
 'position_210': {'P': 93.90243902439023},
 'position_229': {'F': 95.1219512195122},
 'position_240': {'N': 92.6829268292683},
 'position_315': {'W': 90.2439024390244}}